# [SETUP] 
connect to DuckDB

In [1]:
# Load the extension
%load_ext sql

There's a new jupysql version available (0.10.17), you're running 0.10.12. To upgrade: pip install jupysql --upgrade
Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [2]:
# Connect to DuckDB
%sql duckdb:///../../tpch.db

Connecting to 'duckdb:///../../tpch.db'

In [3]:
# set default row display limit
%config SqlMagic.displaylimit = 10

In [4]:
%%sql
-- Run a simple show tables
SELECT
  table_name
FROM
  information_schema.tables
WHERE
  table_schema = 'main'

Running query in 'duckdb:///../../tpch.db'

table_name
customer
lineitem
nation
orders
part
partsupp
region
supplier


In [ ]:
# If you do not see any tables run the below command (after uncommeting it)
#! python setup.py

# [What] Use window functions to do looping with SQL

The reason for window functions is because historically, SQL has been set based i.e supporting operating on a set of rows at a time (e.g. GROUP BY,).

 

Lets see why we may need window functions as opposed to a `GROUP BY`.

![GROUP BY](../../images/groupby.png)

![WINDOW FUNCTION](../../images/window.png)

**NOTE** Notice how `GROUP BY` changes granularity, i.e. the input data had one row per order (aka order grain or order level) but the output had one row per date (aka date grain or date level).

When you want to operate on a set of rows without changing the grain `Window functions` are almost always a good fit.


# [WHY] Common scenarios when you want to use window functions

1. Calculate running metrics/sliding window over rows in the table (aggregate functions)
2. Ranking rows based on values in column(s) (ranking functions)
3. Access other row's values while operating on the current row (value functions)
4. Any combination of the above

## [HOW] to create a window function



### Window functions have four essential parts

1. **Partition**: Defines a set of rows based on specified column(s) value. If no partition is specified, the entire table is considered a partition.
2. **Order By**: This optional clause specifies how to order the rows within a partition.
3. **Function**: The function to be applied on the current row. The function results in an additional column in the output.
4. **Window frame**: Within a partition, a window frame allows you to specify the rows to be considered in the function computation.


![](../../images/framing.svg)

![Create window function](../../images/create_window.png)


In [6]:
%%sql
SELECT
  o_custkey,
  o_orderdate,
  o_totalprice,
  SUM(o_totalprice) -- FUNCTION 
  OVER (
    PARTITION BY
      o_custkey -- PARTITION
    ORDER BY
      o_orderdate -- ORDER BY; ASCENDING ORDER unless specified as DESC
  ) AS running_sum
FROM
  orders
WHERE
  o_custkey = 4
ORDER BY
  o_orderdate

Running query in 'duckdb:///../../tpch.db'

o_custkey,o_orderdate,o_totalprice,running_sum
4,1992-04-26,311722.87,311722.87
4,1992-09-20,182956.01,494678.88
4,1993-10-04,88317.19,582996.07
4,1994-06-10,40347.48,623343.55
4,1995-05-06,104934.65,728278.20
4,1995-11-01,277493.04,1005771.24
4,1996-01-03,46227.94,1051999.18
4,1996-01-06,225294.26,1277293.44
4,1996-06-03,143971.54,1421264.98
4,1996-06-06,235621.83,1656886.81


The function `SUM` that we use in the above query is an aggregate function. Notice how the `running_sum` adds up (aka aggregates) the `o_totalprice` over all the rows. The rows themselves are ordered in ascending order by its orderdate.

**Reference**: While the standard aggregate functions are `MIN, MAX, AVG, SUM, & COUNT`, modern data systems offer a variety of powerful aggregations functions. Check your database documentation for available aggreagate functions. [e.g. list of agg functions available in duckdb](https://duckdb.org/docs/sql/functions/aggregates.html#general-aggregate-functions)

### [Exercise] 

Write a query to calculate the daily running average of totalprice of every customer. 

Time limit during live workshop: 5 min

**Hint**: Figure out the `PARTITION BY` column first, then the `ORDER BY` column and finally the `FUNCTION` to use to compute running average.


In [7]:
%%sql
SELECT
  o_custkey,
  o_orderdate,
  o_totalprice,
  AVG(o_totalprice)
  OVER (
    PARTITION BY
      o_custkey
    ORDER BY
      o_orderdate 
  ) AS running_avg
FROM
  orders
WHERE
  o_custkey = 4
ORDER BY
  o_orderdate

Running query in 'duckdb:///../../tpch.db'

o_custkey,o_orderdate,o_totalprice,running_avg
4,1992-04-26,311722.87,311722.87
4,1992-09-20,182956.01,247339.44
4,1993-10-04,88317.19,194332.02333333335
4,1994-06-10,40347.48,155835.8875
4,1995-05-06,104934.65,145655.64
4,1995-11-01,277493.04,167628.54
4,1996-01-03,46227.94,150285.59714285715
4,1996-01-06,225294.26,159661.68
4,1996-06-03,143971.54,157918.33111111113
4,1996-06-06,235621.83,165688.681


## [HOW] to use ranking functions to get top/bottom n rows

If you are working on a problem to get the top/bottom n rows (as defined by some value) then use the **row** functions.

Let's look at an example of how to use a row function:

From the `orders` table **get the top 3 spending customers per day**. The orders table schema is shown below:

![Orders table](../../images/orders.png)


### [Example]

In [8]:
%%sql
SELECT
  *
FROM
  (
    SELECT
      o_orderdate,
      o_totalprice,
      o_custkey,
      RANK() -- RANKING FUNCTION 
      OVER (
        PARTITION BY
          o_orderdate -- PARTITION BY order date
        ORDER BY
          o_totalprice DESC -- ORDER rows withing partition by totalprice
      ) AS rnk
    FROM
      orders
  )
WHERE
  rnk <= 3
ORDER BY
  o_orderdate
LIMIT
  5;

Running query in 'duckdb:///../../tpch.db'

o_orderdate,o_totalprice,o_custkey,rnk
1992-01-01,464569.20,23122,1
1992-01-01,408062.30,44632,2
1992-01-01,387570.61,67691,3
1992-01-02,429685.09,118874,1
1992-01-02,405608.69,28190,2


### [Exercise] From the `orders` table get the 3 lowest spending customers per day 

**Time limit during live workshop: 5 min**

**Hint**
  1. Figure out the `PARTITION BY` column first, then the `ORDER BY` column and finally the `FUNCTION` to use to compute running average.

The orders table schema is shown below:

![Orders table](../../images/orders.png)


In [9]:
%%sql
SELECT
  *
FROM
  (
    SELECT
      o_orderdate,
      o_totalprice,
      o_custkey,
      RANK() -- RANKING FUNCTION 
      OVER (
        PARTITION BY
          o_orderdate -- PARTITION BY order date
        ORDER BY
          o_totalprice -- ORDER by totalprice ascending
      ) AS rnk
    FROM
      orders
  )
WHERE
  rnk <= 3
ORDER BY
  o_orderdate;

Running query in 'duckdb:///../../tpch.db'

o_orderdate,o_totalprice,o_custkey,rnk
1992-01-01,1324.24,113264,1
1992-01-01,1611.56,109165,2
1992-01-01,2330.74,61658,3
1992-01-02,1663.13,38146,1
1992-01-02,2069.36,7148,2
1992-01-02,2809.42,33319,3
1992-01-03,928.72,44681,1
1992-01-03,1778.25,56062,2
1992-01-03,11230.54,52334,3
1992-01-04,1200.70,103849,1



## [REFERENCE] standard RANKING functions: 

1. **`RANK`**: Ranks the rows starting from 1 to n within the window frame. Ranks the rows with the same value (defined by the "ORDER BY" clause) as the same and skips the ranking numbers that would have been present if the values were different.

2. **`DENSE_RANK`**: Ranks the rows starting from 1 to n within the window frame. Ranks the rows with the same value (defined by the "ORDER BY" clause) as the same and does not skip any ranking numbers.

3. **`ROW_NUMBER`**: Adds a row number that starts from 1 to n within the window frame and does not create any repeating values.



### [Example]

In [10]:
%%sql
-- Let's look at an example showing the difference between RANK, DENSE_RANK and ROW_NUMBER
SELECT 
    order_date,
    order_id,
    total_price,
    ROW_NUMBER() OVER (PARTITION BY order_date ORDER BY total_price) AS row_number,
    RANK() OVER (PARTITION BY order_date ORDER BY total_price) AS rank,
    DENSE_RANK() OVER (PARTITION BY order_date ORDER BY total_price) AS dense_rank
FROM (
    SELECT 
        '2024-07-08' AS order_date, 'order_1' AS order_id, 100 AS total_price UNION ALL
    SELECT 
        '2024-07-08', 'order_2', 200 UNION ALL
    SELECT 
        '2024-07-08', 'order_3', 150 UNION ALL
    SELECT 
        '2024-07-08', 'order_4', 90 UNION ALL
    SELECT 
        '2024-07-08', 'order_5', 100 UNION ALL
    SELECT 
        '2024-07-08', 'order_6', 90 UNION ALL
    SELECT 
        '2024-07-08', 'order_7', 100 UNION ALL
    SELECT 
        '2024-07-10', 'order_8', 100 UNION ALL
    SELECT 
        '2024-07-10', 'order_9', 100 UNION ALL
    SELECT 
        '2024-07-10', 'order_10', 100 UNION ALL
    SELECT 
        '2024-07-11', 'order_11', 100
) AS orders
ORDER BY order_date, row_number;


Running query in 'duckdb:///../../tpch.db'

order_date,order_id,total_price,row_number,rank,dense_rank
2024-07-08,order_6,90,1,1,1
2024-07-08,order_4,90,2,1,1
2024-07-08,order_1,100,3,3,2
2024-07-08,order_7,100,4,3,2
2024-07-08,order_5,100,5,3,2
2024-07-08,order_3,150,6,6,3
2024-07-08,order_2,200,7,7,4
2024-07-10,order_10,100,1,1,1
2024-07-10,order_9,100,2,1,1
2024-07-10,order_8,100,3,1,1


Now that we have see how to **define a window function** and how to use **ranking and aggregation** functions, let's take it a step further by practicing **value functions**.

Remember that value functions are used to access other row's values while operating on the current row

Let's take a look at LEAD and LAG functions:

![LAG AND LEAD](../../images/leadlag.png)

### [Exercise] Write a SQL query using the `orders` table that calculates the following columns:
	1. o_orderdate: From orders table
	2. o_custkey: From orders table
	3. o_totalprice: From orders table
	4. totalprice_diff: The customers current day's o_totalprice - that same customers most recent previous purchase's o_totalprice

* **Time limit during live workshop: 5 min**

* **Hint**: 
    1. Start by figuring out what the `PARTITION BY` column should be, then what the `ORDER BY` column should be, and then finally the function to use. 
    2. Use the `LAG(column_name)` ranking function to identify the prior day's revenue.

* The orders table schema is shown below:

![Orders table](../../images/orders.png)


In [23]:
%%sql

WITH orders_with_rank AS
  (
    SELECT
      o_orderdate,
      o_custkey,
      o_totalprice as current_price,
      o_totalprice - LAG(o_totalprice) 
      OVER (
        PARTITION BY o_custkey
        ORDER BY o_orderdate
      ) AS totalprice_diff
    FROM
      orders
  )

SELECT * 
FROM orders_with_rank
WHERE o_custkey = 4
ORDER BY o_orderdate, o_custkey

Running query in 'duckdb:///../../tpch.db'

o_orderdate,o_custkey,current_price,totalprice_diff
1992-04-26,4,311722.87,None
1992-09-20,4,182956.01,-128766.86
1993-10-04,4,88317.19,-94638.82
1994-06-10,4,40347.48,-47969.71
1995-05-06,4,104934.65,64587.17
1995-11-01,4,277493.04,172558.39
1996-01-03,4,46227.94,-231265.10
1996-01-06,4,225294.26,179066.32
1996-06-03,4,143971.54,-81322.72
1996-06-06,4,235621.83,91650.29


## [REFERENCE] Here is a quick definition of the standard VALUE functions: 

1. **NTILE(n)**: Divides the rows in the window frame into n approximately equal groups, and assigns a number to each row indicating which group it belongs to.
2. **FIRST_VALUE()**: Returns the first value in the window frame.
3. **LAST_VALUE()**: Returns the last value in the window frame.
4. **LAG()**: Accesses data from a previous row within the window frame.
5. **LEAD()**: Accesses data from a subsequent row within the window frame.


# [WHY] define a window frame when you have a partition?

While our functions operate on the rows in the partition a window frame provides more granular ways to operate on a select set of rows within a partition.

When we need to operate one a set of rows within a parition (e.g. a sliding window) we can use the window frame to define these set of rows.

![Three order Sliding window average](../../images/slidingwindow.png)




Example

Consider a scenario where you have sales data, and you want to calculate a 3-day moving average of sales within each store:

```sql
SELECT
    store_id,
    sale_date,
    sales_amount,
    AVG(sales_amount) OVER (
        PARTITION BY store_id
        ORDER BY sale_date
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS moving_avg_sales
FROM
    sales;
```

In this example:

1. **PARTITION BY** store_id ensures the calculation is done separately for each store.
2. **ORDER BY** sale_date defines the order of rows within each partition.
3. **ROWS BETWEEN 2 PRECEDING AND CURRENT ROW** specifies the window frame, considering the current row and the two preceding rows to calculate the moving average.
add: image
Without defining the window frame, the function might not be able to provide the specific moving average calculation you need.

## [HOW] to define a window frame with ROWS

1. **ROWS**: Used to select a set of rows relative to the current row based on position.
    1. Row definition format `ROWS BETWEEN start_point AND end_point`.
    2. The start_point and end_point can be any of the following three (in the proper order:
        1. **n PRECEDING**: n rows preceding the current row. UNBOUNDED PRECEDING indicates all rows before the current row.
        2. **n FOLLOWING**: n rows following the current row. UNBOUNDED FOLLOWING indicates all rows after the current row.


Let's see how relative row numbers can be used to define a window range.

Consider this window function 

```sql
AVG(total_price) OVER ( -- FUNCTION: RUNNING AVERAGE
    PARTITION BY o_custkey -- PARTITIONED BY customer
    ORDER BY order_month 
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING -- WINDOW FRAME DEFINED AS 1 ROW PRECEDING to 1 ROW FOLLOWING
    )
```

![Window frame with ROWS](../../images/wf.png)

### [Example]


1. Write a query on the orders table that has the following output:
    1. o_custkey
    2. order_month: In YYYY-MM format, use strftime(o_orderdate, '%Y-%m') AS order_month
    3. total_price: Sum of o_totalprice for that month
    4. three_mo_total_price_avg: The 3 month (previous, current & next) average of total_price for that customer



In [36]:
%%sql
SELECT
  order_month,
  o_custkey,
  total_price,
  ROUND(
    AVG(total_price) OVER ( -- FUNCTION: RUNNING AVERAGE
      PARTITION BY
        o_custkey -- PARTITIONED BY customer
      ORDER BY
        order_month ROWS BETWEEN 1 PRECEDING
        AND 1 FOLLOWING -- WINDOW FRAME DEFINED AS 1 ROW PRECEDING to 1 ROW FOLLOWING
    ),
    2
  ) AS three_mo_total_price_avg
FROM
  (
    SELECT
      strftime (o_orderdate, '%Y-%m') AS order_month,
      o_custkey,
      SUM(o_totalprice) AS total_price
    FROM
      orders
    GROUP BY
      1,
      2
  )
LIMIT
  5;

Running query in 'duckdb:///../../tpch.db'

order_month,o_custkey,total_price,three_mo_total_price_avg
1992-06,11,143292.53,106449.83
1993-06,11,69607.13,137299.5
1993-08,11,198998.83,166972.36
1993-09,11,232311.12,205324.1
1993-11,11,184662.35,180752.97


## [HOW] to define a window frame with RANGE

1. **RANGE**: Used to select a set of rows relative to the current row based on the value of the columns specified in the `ORDER BY` clause.
    1. Range definition format `RANGE BETWEEN start_point AND end_point`.
    2. The start_point and end_point can be any of the following:
        1. **CURRENT ROW**: The current row.
        2. **n PRECEDING**: All rows with values within the specified range that are less than or equal to n units preceding the value of the current row.
        3. **n FOLLOWING**: All rows with values within the specified range that are greater than or equal to n units following the value of the current row.
        4. **UNBOUNDED PRECEDING**: All rows before the current row within the partition.
        5. **UNBOUNDED FOLLOWING**: All rows after the current row within the partition.
    3. `RANGE` is particularly useful when dealing with numeric or date/time ranges, allowing for calculations like running totals, moving averages, or cumulative distributions.

Let's see how `RANGE` works with `AVG(total price) OVER (PARTITION BY customer id ORDER BY date RANGE BETWEEN INTERVAL '1' DAY PRECEDING AND '1' DAY FOLLOWING)` using the below visualization:

![RANGE](../../images/range.png)

### [Exercise]

Now that we have seen how to create a window frame with ROWS, let' explore how to do this with RANGE.

1. Write a query on the orders table that has the following output:
    1. order_month, 
    2. o_custkey,
    3. total_price,
    4. three_mo_total_price_avg
    5. **consecutive_three_mo_total_price_avg**: The consecutive 3 month average of total_price for that customer. Note that this should only include months that are chronologically next to each other.
                                                                                               
**Time limit during live workshop: 10 min**
                                                                                               
**Hint**: Use `CAST(strftime(o_orderdate, '%Y-%m-01') AS DATE)` to cast order_month to date format.

**Hint**: Use the `INTERVAL` format shown above to construct the window function to compute `consecutive_three_mo_total_price_avg` column.
                                                                                           

In [38]:
%%sql
SELECT
  order_month,
  o_custkey,
  total_price,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        CAST(order_month AS DATE) RANGE BETWEEN INTERVAL '1' MONTH PRECEDING
        AND INTERVAL '1' MONTH FOLLOWING
    ),
    2
  ) AS consecutive_three_mo_total_price_avg,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        order_month ROWS BETWEEN 1 PRECEDING
        AND 1 FOLLOWING
    ),
    2
  ) AS three_mo_total_price_avg
FROM
  (
    SELECT
      strftime (o_orderdate, '%Y-%m-01') AS order_month,
      o_custkey,
      SUM(o_totalprice) AS total_price
    FROM
      orders
    GROUP BY
      1,
      2
  )
ORDER BY
  o_custkey,
  order_month
LIMIT
  50;

Running query in 'duckdb:///../../tpch.db'

order_month,o_custkey,total_price,consecutive_three_mo_total_price_avg,three_mo_total_price_avg
1992-04-01,1,74602.81,74602.81,98839.83
1992-08-01,1,123076.84,123076.84,87719.23
1996-06-01,1,65478.05,120062.0,121066.94
1996-07-01,1,174645.94,120062.0,98057.42
1996-12-01,1,54048.26,54048.26,108201.74
1997-03-01,1,95911.01,95911.01,74979.63
1992-04-01,2,167016.61,167016.61,135157.14
1994-05-01,2,103297.68,103297.68,95603.21
1994-08-01,2,16495.33,16495.33,50958.61
1994-12-01,2,33082.83,33082.83,90325.17


# Recap

1. Use windows when:
	* Calculating running metrics (similar to GROUP BY, but keeping all the rows)
	* Rank rows based on specific column(s)
	* Access other row values from the current row
2. Window has four key parts: Partition, Order By, Function, Window Frame
3. Define window frame with ROWS or RANGE
4. Window functions are expensive; be mindful of the performance (Covered in another lesson)

# Helpers

1. Solutions are available at [workshop_solutions](./workshop_solutions.ipynb). **Note** You need to stop the kernel in this notebook before starting the next one, since DuckDB can only have one worker on it at a time.
2. Note the `outline`(or `Table of Contents` in the left pane on Jupyter notebook) is a easy way to navigate this workbook.

# Questions


In [ ]:
exit

Closing duckdb:///../../tpch.db

: 